In [1]:
import os
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy
from keras.regularizers import l2

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DATA_DIR = 'data'
HDF5_FILE = 'data_tf.hdf5'

In [4]:
BATCH_SIZE = 64
IMG_SHAPE = (96, 96, 4)

In [5]:
data_file = os.path.join(DATA_DIR, HDF5_FILE)
ds_x = tfio.IODataset.from_hdf5(data_file, '/x_train')
ds_y = tfio.IODataset.from_hdf5(data_file, '/y_train')

In [6]:
# convert to float and scale to [0, 1)
ds_x = ds_x.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
# ds_x = ds_images.map(lambda x: tf.image.per_image_standardization(x))

In [7]:
ds_xy = tf.data.Dataset.zip((ds_x, ds_y))
ds_xy = ds_xy.shuffle(BATCH_SIZE*2)

ds_len = ds_xy.reduce(0, lambda x, _: x + 1).numpy()
train_len = int(0.9 * ds_len)

ds_train = ds_xy.take(train_len)
ds_test = ds_xy.skip(train_len)

ds_len, train_len

(220025, 198022)

In [8]:
model = Sequential()

model.add(Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(1e-5))) #, input_shape=(X_train.shape[1:])))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(1e-5)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(32, (3,3), activation='relu', kernel_regularizer=l2(1e-5)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), activation='relu', kernel_regularizer=l2(1e-5)))
model.add(BatchNormalization())
model.add(MaxPooling2D (pool_size=(2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(16, (3,3), activation='relu', kernel_regularizer=l2(1e-5)))
model.add(BatchNormalization())
model.add(Conv2D(16, (3,3), activation='relu', kernel_regularizer=l2(1e-5)))
model.add(BatchNormalization())
model.add(MaxPooling2D (pool_size=(2, 2), strides=(1, 1)))
# model.add(Dropout(0.4))
model.add(Dropout(0.6))

model.add(Flatten())
# model.add(Dropout(0.2))

# model.add(Dense(64, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.4))

model.add(Dense(1, activation='sigmoid'))

# model.summary()

In [9]:
ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
3095/3095 - 192s - loss: 0.4239 - accuracy: 0.8273 - val_loss: 0.4719 - val_accuracy: 0.8224
Epoch 2/10
3095/3095 - 188s - loss: 0.2973 - accuracy: 0.8797 - val_loss: 0.9271 - val_accuracy: 0.7554
Epoch 3/10
3095/3095 - 185s - loss: 0.2612 - accuracy: 0.8974 - val_loss: 0.4064 - val_accuracy: 0.8545
Epoch 4/10
3095/3095 - 186s - loss: 0.2387 - accuracy: 0.9077 - val_loss: 1.2922 - val_accuracy: 0.7327
Epoch 5/10
3095/3095 - 183s - loss: 0.2225 - accuracy: 0.9152 - val_loss: 1.4469 - val_accuracy: 0.7249
Epoch 6/10
3095/3095 - 183s - loss: 0.2148 - accuracy: 0.9194 - val_loss: 0.2301 - val_accuracy: 0.9118
Epoch 7/10
3095/3095 - 183s - loss: 0.2089 - accuracy: 0.9219 - val_loss: 0.9130 - val_accuracy: 0.7589
Epoch 8/10
3095/3095 - 183s - loss: 0.2045 - accuracy: 0.9253 - val_loss: 0.8725 - val_accuracy: 0.7677
Epoch 9/10
3095/3095 - 183s - loss: 0.1981 - accuracy: 0.9279 - val_loss: 0.4767 - val_accuracy: 0.8508
Epoch 10/10
3095/3095 - 182s - loss: 0.1952 - accuracy: 0.9292 -

In [10]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=2e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
1548/1548 - 193s - loss: 0.1640 - accuracy: 0.9428 - val_loss: 0.1526 - val_accuracy: 0.9480
Epoch 2/10
1548/1548 - 190s - loss: 0.1593 - accuracy: 0.9444 - val_loss: 0.1558 - val_accuracy: 0.9473
Epoch 3/10
1548/1548 - 189s - loss: 0.1570 - accuracy: 0.9453 - val_loss: 0.2495 - val_accuracy: 0.9126
Epoch 4/10
1548/1548 - 189s - loss: 0.1560 - accuracy: 0.9461 - val_loss: 0.1467 - val_accuracy: 0.9505
Epoch 5/10
1548/1548 - 180s - loss: 0.1552 - accuracy: 0.9462 - val_loss: 0.1498 - val_accuracy: 0.9501
Epoch 6/10
1548/1548 - 189s - loss: 0.1526 - accuracy: 0.9469 - val_loss: 0.1591 - val_accuracy: 0.9451
Epoch 7/10
1548/1548 - 190s - loss: 0.1528 - accuracy: 0.9469 - val_loss: 0.1550 - val_accuracy: 0.9478
Epoch 8/10
1548/1548 - 190s - loss: 0.1511 - accuracy: 0.9474 - val_loss: 0.1478 - val_accuracy: 0.9511
Epoch 9/10
1548/1548 - 189s - loss: 0.1504 - accuracy: 0.9479 - val_loss: 0.1713 - val_accuracy: 0.9405
Epoch 10/10
1548/1548 - 189s - loss: 0.1497 - accuracy: 0.9481 -

In [11]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
774/774 - 199s - loss: 0.1419 - accuracy: 0.9514 - val_loss: 0.1395 - val_accuracy: 0.9543
Epoch 2/10
774/774 - 185s - loss: 0.1414 - accuracy: 0.9519 - val_loss: 0.1380 - val_accuracy: 0.9551
Epoch 3/10
774/774 - 184s - loss: 0.1400 - accuracy: 0.9520 - val_loss: 0.1401 - val_accuracy: 0.9536
Epoch 4/10
774/774 - 194s - loss: 0.1404 - accuracy: 0.9516 - val_loss: 0.1368 - val_accuracy: 0.9553
Epoch 5/10
774/774 - 185s - loss: 0.1402 - accuracy: 0.9520 - val_loss: 0.1384 - val_accuracy: 0.9556
Epoch 6/10
774/774 - 196s - loss: 0.1395 - accuracy: 0.9522 - val_loss: 0.1384 - val_accuracy: 0.9557
Epoch 7/10
774/774 - 185s - loss: 0.1393 - accuracy: 0.9527 - val_loss: 0.1392 - val_accuracy: 0.9559
Epoch 8/10
774/774 - 194s - loss: 0.1389 - accuracy: 0.9528 - val_loss: 0.1391 - val_accuracy: 0.9553
Epoch 9/10
774/774 - 195s - loss: 0.1390 - accuracy: 0.9528 - val_loss: 0.1396 - val_accuracy: 0.9546
Epoch 10/10
774/774 - 195s - loss: 0.1392 - accuracy: 0.9526 - val_loss: 0.1430 - 

In [12]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
774/774 - 194s - loss: 0.1375 - accuracy: 0.9531 - val_loss: 0.1382 - val_accuracy: 0.9544
Epoch 2/10
774/774 - 194s - loss: 0.1376 - accuracy: 0.9531 - val_loss: 0.1389 - val_accuracy: 0.9548
Epoch 3/10
774/774 - 194s - loss: 0.1378 - accuracy: 0.9531 - val_loss: 0.1370 - val_accuracy: 0.9556
Epoch 4/10
774/774 - 196s - loss: 0.1374 - accuracy: 0.9529 - val_loss: 0.1387 - val_accuracy: 0.9551
Epoch 5/10
774/774 - 190s - loss: 0.1378 - accuracy: 0.9533 - val_loss: 0.1375 - val_accuracy: 0.9546
Epoch 6/10
774/774 - 197s - loss: 0.1378 - accuracy: 0.9531 - val_loss: 0.1384 - val_accuracy: 0.9551
Epoch 7/10
774/774 - 185s - loss: 0.1367 - accuracy: 0.9536 - val_loss: 0.1380 - val_accuracy: 0.9550
Epoch 8/10
774/774 - 186s - loss: 0.1382 - accuracy: 0.9529 - val_loss: 0.1377 - val_accuracy: 0.9547
Epoch 9/10
774/774 - 195s - loss: 0.1376 - accuracy: 0.9528 - val_loss: 0.1376 - val_accuracy: 0.9553
Epoch 10/10
774/774 - 190s - loss: 0.1378 - accuracy: 0.9534 - val_loss: 0.1372 - 